In [6]:
### Part 1 ###

def get_input_lines():
    with open('day13_input.txt') as f:
        return [x for x in f.read().splitlines()]

input_lines = get_input_lines()
eta = int(input_lines[0])
bus_str_list = input_lines[1].split(',')

def buses_str_to_list(bus_str_list: str):
    buses_list = []
    for bus_str in bus_str_list:
        if bus_str == 'x':
            continue
        buses_list.append(int(bus_str))
    return buses_list

buses_list = buses_str_to_list(bus_str_list)

# Algo:
# 1. Do ((eta // bus_num) + 1 )* bus_num - this will be the first bus to get
# 2. minus off eta
# 3. find index of smallest element

def find_wait_time_for_bus(bus: int):
    return (((eta // bus) + 1 )* bus) - eta

wait_times = map(find_wait_time_for_bus, buses_list)

min_wait_time, index = min((min_wait_time, index) for (index, min_wait_time) in enumerate(wait_times))

best_bus = buses_list[index]
print(f'Min wait time: {min_wait_time}')
print(f'Best bus: {best_bus}')
print(f'Product: {min_wait_time * best_bus}')


Min wait time: 6
Best bus: 383
Product: 2298


In [17]:
### Part 2 ##

# Sneaky bit of chinese remainder theorem
# Assuming all bus_names are primes, and we call them n_1, n_2 ... n_k, then we must find t s.t:
# t ~ 0 mod n_1
# t + 1 ~ 0 mod n_2, so t ~ - 1 ~ n_2 - 1 mod n_2
# ...
# so the pattern is: t ~ n_i - i mod n_i

# STEP 1: Collect up the list of 'Chinese Remainders'

def get_input_lines():
    with open('day13_input.txt') as f:
        return [x for x in f.read().splitlines()]

input_lines = get_input_lines()
bus_str_list = input_lines[1].split(',')

class ChineseRemainder():
    def __init__(self, remainder: int, modulus: int):
        self.rem = remainder
        self.mod = modulus

chinese_remainder_list = []
for index, bus_str in enumerate(bus_str_list):
    if (bus_str != 'x'):
        mod = int(bus_str)
        chinese_remainder_list.append(ChineseRemainder(((mod - index) % mod), mod))

for c_r in chinese_remainder_list:
    print(f'Rem: {c_r.rem}. Mod: {c_r.mod}.')


# STEP 2: Set up function to find Bézout coefficients using Extended Euclidean Algo

def bezout_coeffs(a: int, b: int):
    # returns (s, t) such that a*s + b*t = gcd(a,b)
    #   (always = 1 in this task, but can work in general too)

    old_r, r = a, b
    old_s, s = 1, 0
    old_t, t = 0, 1

    while r != 0:
        quotient = old_r // r
        old_r, r = r, (old_r - (quotient * r))
        old_s, s = s, (old_s - (quotient * s))
        old_t, t = t, (old_t - (quotient * t))
    
    return old_s, old_t

# STEP 3: Do pair-wise solving of CRT (e.g. solve 1&2, then solve (1&2)&3 etc)

c_r_1 = chinese_remainder_list[0]

for c_r_2 in chinese_remainder_list[1:]:
    # 1. Find Bézout coeffs of the mods
    m_1, m_2 = bezout_coeffs(c_r_1.mod, c_r_2.mod)
    
    # 2. Plug into special formula to find out solution (see CRT on Wiki)
    solution = (c_r_1.rem*c_r_2.mod*m_2 + c_r_2.rem*c_r_1.mod*m_1) % (c_r_1.mod*c_r_2.mod)
    c_r_1 = ChineseRemainder(solution, c_r_1.mod*c_r_2.mod)

print(c_r_1.rem)


Rem: 0. Mod: 23.
Rem: 28. Mod: 41.
Rem: 360. Mod: 383.
Rem: 3. Mod: 13.
Rem: 14. Mod: 17.
Rem: 15. Mod: 19.
Rem: 6. Mod: 29.
Rem: 449. Mod: 503.
Rem: 20. Mod: 37.
783685719679632
